In [2]:
# Instalação de dependências
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Importação e Preparação do Dataset
import json
import pandas as pd
from datasets import load_dataset


In [8]:
# Carregar o JSON diretamente usando streaming
file_path = "/content/drive/MyDrive/fiap/tech-challenge-03/trn.json"

# Carregar o dataset com as colunas "title" e "content"
dataset = load_dataset("json", data_files=file_path, split="train", streaming=True)

# Função de processamento para formatar os prompts e respostas
def format_dataset(example):
    """
    Formata cada exemplo do dataset para fine-tuning.
    """
    # Remover linhas com dados ausentes
    if example["title"] and example["content"]:
        return {
            "prompt": f"Pergunta: Descreva o produto | Contexto: {example['title']}",
            "response": example["content"]
        }
    # Return an empty dictionary if title or content is missing to avoid None values
    return {}

# Filtrar exemplos com valores não nulos em "title" e "content"
dataset = dataset.filter(lambda example: example["title"] is not None and example["content"] is not None)

# Aplicar transformação no dataset
formatted_dataset = dataset.map(format_dataset, remove_columns=["title", "content"])

In [11]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# Carregar o tokenizer e modelo GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Tokenizar o dataset
def tokenize_function(examples):
    """
    Tokeniza os exemplos no modo batched.
    """
    inputs = [
        example["prompt"] + "\nResposta: " + example["response"]
        for example in zip(examples["prompt"], examples["response"])
    ]
    return tokenizer(inputs, truncation=True, padding="max_length", max_length=512)

tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True, remove_columns=["prompt", "response"])

num_examples = sum(1 for _ in dataset)
print(f"Total de exemplos: {num_examples}")

# Calcule os passos
batch_size = 4
max_steps = num_examples // batch_size
print(f"Passos necessários: {max_steps}")

# Configuração do treinamento
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    max_steps=max_steps,  # Especifique o número de passos manualmente
    logging_dir="./logs",
    logging_steps=100,
    fp16=True,
    report_to="none"
)


# Inicializar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

# Iniciar o treinamento
trainer.train()


max_steps is given, it will override any value given in num_train_epochs


Total de exemplos: 2248619
Passos necessários: 562154


TypeError: tuple indices must be integers or slices, not str

In [ ]:
# Avaliação e Geração de Respostas
def generate_response(question, title):
    input_text = f"Pergunta: {question} | Contexto: {title}"
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Teste do modelo treinado
question = "Qual é a descrição do produto?"
title = "Laptop Dell Inspiron 15"
print(generate_response(question, title))